In [1]:
%matplotlib inline
import os, sys
import numpy as np
from utils_data_exploration import *
import gc

### Directory Structure

In [2]:
%pwd # verify you are in the correct folder

'/home/javier/Documents/YelpRestaurantPhotoClassification/nbs'

In [3]:
dataset_folder = 'imagenet/'

In [4]:
# Assign Relevant General Paths, i.e. dataset invariant paths
LESSON_HOME_DIR = os.getcwd() # path tho the notebook
DATA_DIR = LESSON_HOME_DIR + '/../data/'
TRAIN_PATH = DATA_DIR + '/train_photos/'
VALID_PATH = DATA_DIR + '/valid_photos/'
SHARED_PATH = DATA_DIR + 'shared/'

# Assign Specific Paths, i.e. dataset specific paths
RESULTS_PATH = DATA_DIR + '/results/' + dataset_folder

FEATURES_PATH = RESULTS_PATH + 'features/'
CLASSIFIERS_PATH = RESULTS_PATH + 'classifiers/'

# Train Set

### Start with the list of business and the FC1 Buisness (see Extract_F1_Features.ipynb)

In [5]:
businesses = np.load(SHARED_PATH+'/businesses_shuffled.npy')
businesses_fc1_features = np.load(FEATURES_PATH + 'businesses_fc1_blueprint.npy')

### Divide the restaurants in 4-folds for Cross Validation

In [6]:
nb_folds = 4

folds = dict()
total_restaurants = len(businesses)
restaurants_per_fold = int(len(businesses) / nb_folds)
businesses_idx = np.arange(total_restaurants)

for fold in range(nb_folds):
    folds[fold] = businesses_idx[fold*restaurants_per_fold:(fold+1)*restaurants_per_fold]

### Retrieve the binary labels (apply / does'nt apply) for every attribute

In [7]:
attributes = 9 # possible restaurante attributes
labels_dict = dict()
nb_businesses = len(businesses)

for attribute in range(attributes):
    labels_dict[attribute] = np.zeros(nb_businesses)
    for i, business in enumerate(businesses):
        if attribute in df_train_labels.loc[business]['labels']:
            labels_dict[attribute][i] = 1

In [8]:
labels_dict

{0: array([ 1.,  0.,  1., ...,  1.,  0.,  0.]),
 1: array([ 1.,  1.,  0., ...,  0.,  1.,  1.]),
 2: array([ 0.,  1.,  1., ...,  0.,  1.,  0.]),
 3: array([ 1.,  1.,  0., ...,  1.,  1.,  0.]),
 4: array([ 0.,  1.,  0., ...,  0.,  0.,  0.]),
 5: array([ 1.,  1.,  0., ...,  0.,  1.,  0.]),
 6: array([ 1.,  1.,  1., ...,  0.,  1.,  0.]),
 7: array([ 0.,  1.,  0., ...,  0.,  0.,  0.]),
 8: array([ 0.,  0.,  1., ...,  1.,  0.,  1.])}

# Train classifiers on top of the blueprint bottleneck features per restaurant

In [9]:
from keras import backend as K
from keras import applications
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

Using TensorFlow backend.


### Create a copy of the VGG16 classifier(image net) -> adding dropout and binarizing the output

In [24]:
def get_binary_vgg16_classifier(drop_rate = 0.5):
    
    # Replicate the fc classifier of vgg net
    model = Sequential()
    model.add(Dropout(drop_rate, input_shape=(4096,)))
    model.add(Dense(4096, activation='relu', name='fc2'))
    model.add(Dropout(drop_rate))
    model.add(Dense(1000, activation='relu', name='predictions'))
    
    # Load pre trained weights for image net
    model.load_weights('/home/javier/./.keras/models/vgg16_weights_tf_dim_ordering_tf_kernels.h5', by_name=True)
    
    # Account for the added dropout
    fc2 = model.layers[-3]
    fc2.set_weights([w/drop_rate for w in fc2.get_weights()])
    
    # Remove the predictions layer and add a binary apply/doesen't apply neuron
    model.pop()
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(lr=5e-06), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

## Train 4 classifiers per label, one for each fold

In [24]:
val_fold = 0
train_idx = [fold_idx for i, fold_idx in folds.items() if i != val_fold]

In [25]:
for attribute in range(attributes):
    for val_fold in range(len(folds)):
        
        K.clear_session()
        gc.collect()
        
        # grab a binary classifier model with imagenet weights
        classifier_model = get_binary_vgg16_classifier()
        
        # grab the training-validation data
        train_idx = []
        train_idx = [fold_idx for i, fold_idx in folds.items() if i != val_fold]
        train_idx = np.array([idx for sublist in train_idx for idx in sublist], dtype=np.int32)
        valid_idx = np.array(folds[val_fold], dtype=np.int32)
        X_train = businesses_fc1_features[train_idx, :]; y_train = labels_dict[attribute][train_idx]
        X_valid = businesses_fc1_features[valid_idx, :]; y_valid = labels_dict[attribute][valid_idx]
        
        # Train the prediction layer (100 epochs) without training fc2 weigths
        for layer in classifier_model.layers:
            if layer.name == 'fc2':
                layer.trainable = False
        
        h = classifier_model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=100)
        np.save(RESULTS_PATH+'/history_prediction_attribute_%d_fold_%d'%(attribute, val_fold), h.history)
        
        # Now Train FC2 -> Run for 150 epochs, adam(lr=1e-06)
        for layer in classifier_model.layers:
            if layer.name == 'fc2':
                layer.trainable = True
        
        classifier_model.compile(optimizer=Adam(lr=5e-06), loss='binary_crossentropy', metrics=['accuracy'])
        h = classifier_model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=100)
        np.save(RESULTS_PATH+'/history_fc2_attribute_%d_fold_%d'%(attribute, val_fold), h.history)
        
        classifier_model.save_weights(RESULTS_PATH+'/ft_200_attribute_%d_fold_%d.h5' % (attribute, val_fold))
        
        

Train on 1497 samples, validate on 499 samples
Epoch 1/100
1497/1497 [==============================] - 0s - loss: 0.8676 - acc: 0.5858 - val_loss: 0.5321 - val_acc: 0.7194
Epoch 2/100
1497/1497 [==============================] - 0s - loss: 0.6472 - acc: 0.6807 - val_loss: 0.4763 - val_acc: 0.7515
Epoch 3/100
1497/1497 [==============================] - 0s - loss: 0.5992 - acc: 0.7141 - val_loss: 0.4546 - val_acc: 0.7776
Epoch 4/100
1497/1497 [==============================] - 0s - loss: 0.5902 - acc: 0.7401 - val_loss: 0.4551 - val_acc: 0.7856
Epoch 5/100
1497/1497 [==============================] - 0s - loss: 0.5512 - acc: 0.7395 - val_loss: 0.4415 - val_acc: 0.7976
Epoch 6/100
1497/1497 [==============================] - 0s - loss: 0.5560 - acc: 0.7328 - val_loss: 0.4475 - val_acc: 0.7856
Epoch 7/100
1497/1497 [==============================] - 0s - loss: 0.5548 - acc: 0.7388 - val_loss: 0.4314 - val_acc: 0.8076
Epoch 8/100
1497/1497 [==============================] - 0s - loss: 0.5

1497/1497 [==============================] - 0s - loss: 0.2896 - acc: 0.8851 - val_loss: 0.4019 - val_acc: 0.8176
Epoch 30/100
1497/1497 [==============================] - 0s - loss: 0.3028 - acc: 0.8737 - val_loss: 0.4013 - val_acc: 0.8216
Epoch 31/100
1497/1497 [==============================] - 0s - loss: 0.2775 - acc: 0.8758 - val_loss: 0.4021 - val_acc: 0.8116
Epoch 32/100
1497/1497 [==============================] - 0s - loss: 0.2661 - acc: 0.8918 - val_loss: 0.3991 - val_acc: 0.8216
Epoch 33/100
1497/1497 [==============================] - 0s - loss: 0.2950 - acc: 0.8744 - val_loss: 0.4092 - val_acc: 0.8036
Epoch 34/100
1497/1497 [==============================] - 0s - loss: 0.2793 - acc: 0.8811 - val_loss: 0.4003 - val_acc: 0.8156
Epoch 35/100
1497/1497 [==============================] - 0s - loss: 0.2758 - acc: 0.8878 - val_loss: 0.3984 - val_acc: 0.8196
Epoch 36/100
1497/1497 [==============================] - 0s - loss: 0.2759 - acc: 0.8884 - val_loss: 0.3989 - val_acc: 0.81

1497/1497 [==============================] - 0s - loss: 0.3355 - acc: 0.8544 - val_loss: 0.4640 - val_acc: 0.7856
Epoch 58/100
1497/1497 [==============================] - 0s - loss: 0.3439 - acc: 0.8524 - val_loss: 0.4642 - val_acc: 0.7896
Epoch 59/100
1497/1497 [==============================] - 0s - loss: 0.3430 - acc: 0.8597 - val_loss: 0.4665 - val_acc: 0.7756
Epoch 60/100
1497/1497 [==============================] - 0s - loss: 0.3523 - acc: 0.8517 - val_loss: 0.4685 - val_acc: 0.7916
Epoch 61/100
1497/1497 [==============================] - 0s - loss: 0.3330 - acc: 0.8517 - val_loss: 0.4667 - val_acc: 0.7856
Epoch 62/100
1497/1497 [==============================] - 0s - loss: 0.3433 - acc: 0.8564 - val_loss: 0.4665 - val_acc: 0.7816
Epoch 63/100
1497/1497 [==============================] - 0s - loss: 0.3337 - acc: 0.8624 - val_loss: 0.4672 - val_acc: 0.7776
Epoch 64/100
1497/1497 [==============================] - 0s - loss: 0.3210 - acc: 0.8697 - val_loss: 0.4704 - val_acc: 0.77

1497/1497 [==============================] - 0s - loss: 0.2249 - acc: 0.9158 - val_loss: 0.4911 - val_acc: 0.7996
Epoch 86/100
1497/1497 [==============================] - 0s - loss: 0.2289 - acc: 0.9058 - val_loss: 0.4911 - val_acc: 0.7996
Epoch 87/100
1497/1497 [==============================] - 0s - loss: 0.2262 - acc: 0.9112 - val_loss: 0.4927 - val_acc: 0.7896
Epoch 88/100
1497/1497 [==============================] - 0s - loss: 0.2298 - acc: 0.9105 - val_loss: 0.4934 - val_acc: 0.7856
Epoch 89/100
1497/1497 [==============================] - 0s - loss: 0.2112 - acc: 0.9232 - val_loss: 0.4935 - val_acc: 0.7876
Epoch 90/100
1497/1497 [==============================] - 0s - loss: 0.2170 - acc: 0.9172 - val_loss: 0.5002 - val_acc: 0.7856
Epoch 91/100
1497/1497 [==============================] - 0s - loss: 0.2235 - acc: 0.9138 - val_loss: 0.4986 - val_acc: 0.7856
Epoch 92/100
1497/1497 [==============================] - 0s - loss: 0.2289 - acc: 0.9105 - val_loss: 0.4950 - val_acc: 0.78

1497/1497 [==============================] - 0s - loss: 0.2819 - acc: 0.8818 - val_loss: 0.4473 - val_acc: 0.7996
Epoch 14/100
1497/1497 [==============================] - 0s - loss: 0.2720 - acc: 0.8898 - val_loss: 0.4443 - val_acc: 0.8016
Epoch 15/100
1497/1497 [==============================] - 0s - loss: 0.2795 - acc: 0.8904 - val_loss: 0.4492 - val_acc: 0.7876
Epoch 16/100
1497/1497 [==============================] - 0s - loss: 0.2955 - acc: 0.8711 - val_loss: 0.4392 - val_acc: 0.8036
Epoch 17/100
1497/1497 [==============================] - 0s - loss: 0.2692 - acc: 0.8838 - val_loss: 0.4384 - val_acc: 0.8056
Epoch 18/100
1497/1497 [==============================] - 0s - loss: 0.2847 - acc: 0.8811 - val_loss: 0.4448 - val_acc: 0.8036
Epoch 19/100
1497/1497 [==============================] - 0s - loss: 0.2880 - acc: 0.8791 - val_loss: 0.4374 - val_acc: 0.8016
Epoch 20/100
1497/1497 [==============================] - 0s - loss: 0.2716 - acc: 0.8884 - val_loss: 0.4417 - val_acc: 0.80

...

1497/1497 [==============================] - 0s - loss: 0.2989 - acc: 0.8731 - val_loss: 0.3792 - val_acc: 0.8617
Epoch 58/100
1497/1497 [==============================] - 0s - loss: 0.2742 - acc: 0.8778 - val_loss: 0.3781 - val_acc: 0.8597
Epoch 59/100
1497/1497 [==============================] - 0s - loss: 0.2700 - acc: 0.8871 - val_loss: 0.3798 - val_acc: 0.8557
Epoch 60/100
1497/1497 [==============================] - 0s - loss: 0.2630 - acc: 0.8884 - val_loss: 0.3839 - val_acc: 0.8597
Epoch 61/100
1497/1497 [==============================] - 0s - loss: 0.2597 - acc: 0.8864 - val_loss: 0.3759 - val_acc: 0.8577
Epoch 62/100
1497/1497 [==============================] - 0s - loss: 0.2607 - acc: 0.8878 - val_loss: 0.3762 - val_acc: 0.8557
Epoch 63/100
1497/1497 [==============================] - 0s - loss: 0.2609 - acc: 0.8844 - val_loss: 0.3758 - val_acc: 0.8617
Epoch 64/100
1497/1497 [==============================] - 0s - loss: 0.2706 - acc: 0.8831 - val_loss: 0.3728 - val_acc: 0.85

1497/1497 [==============================] - 0s - loss: 0.1722 - acc: 0.9292 - val_loss: 0.3831 - val_acc: 0.8657
Epoch 86/100
1497/1497 [==============================] - 0s - loss: 0.1793 - acc: 0.9279 - val_loss: 0.3897 - val_acc: 0.8557
Epoch 87/100
1497/1497 [==============================] - 0s - loss: 0.1676 - acc: 0.9359 - val_loss: 0.3807 - val_acc: 0.8717
Epoch 88/100
1497/1497 [==============================] - 0s - loss: 0.1797 - acc: 0.9245 - val_loss: 0.3860 - val_acc: 0.8677
Epoch 89/100
1497/1497 [==============================] - 0s - loss: 0.1761 - acc: 0.9285 - val_loss: 0.3766 - val_acc: 0.8798
Epoch 90/100
1497/1497 [==============================] - 0s - loss: 0.1831 - acc: 0.9172 - val_loss: 0.3809 - val_acc: 0.8657
Epoch 91/100
1497/1497 [==============================] - 0s - loss: 0.1732 - acc: 0.9312 - val_loss: 0.3812 - val_acc: 0.8717
Epoch 92/100
1497/1497 [==============================] - 0s - loss: 0.1741 - acc: 0.9285 - val_loss: 0.3801 - val_acc: 0.86

1497/1497 [==============================] - 0s - loss: 0.2216 - acc: 0.9085 - val_loss: 0.3127 - val_acc: 0.8597
Epoch 14/100
1497/1497 [==============================] - 0s - loss: 0.2200 - acc: 0.9058 - val_loss: 0.3183 - val_acc: 0.8557
Epoch 15/100
1497/1497 [==============================] - 0s - loss: 0.2151 - acc: 0.9125 - val_loss: 0.3176 - val_acc: 0.8577
Epoch 16/100
1497/1497 [==============================] - 0s - loss: 0.2193 - acc: 0.9078 - val_loss: 0.3261 - val_acc: 0.8497
Epoch 17/100
1497/1497 [==============================] - 0s - loss: 0.1997 - acc: 0.9205 - val_loss: 0.3217 - val_acc: 0.8537
Epoch 18/100
1497/1497 [==============================] - 0s - loss: 0.2154 - acc: 0.9112 - val_loss: 0.3321 - val_acc: 0.8517
Epoch 19/100
1497/1497 [==============================] - 0s - loss: 0.2088 - acc: 0.9212 - val_loss: 0.3336 - val_acc: 0.8477
Epoch 20/100
1497/1497 [==============================] - 0s - loss: 0.2265 - acc: 0.9071 - val_loss: 0.3204 - val_acc: 0.85

1497/1497 [==============================] - 0s - loss: 0.2995 - acc: 0.8697 - val_loss: 0.3474 - val_acc: 0.8657
Epoch 42/100
1497/1497 [==============================] - 0s - loss: 0.2855 - acc: 0.8798 - val_loss: 0.3496 - val_acc: 0.8577
Epoch 43/100
1497/1497 [==============================] - 0s - loss: 0.2919 - acc: 0.8784 - val_loss: 0.3469 - val_acc: 0.8617
Epoch 44/100
1497/1497 [==============================] - 0s - loss: 0.2695 - acc: 0.8858 - val_loss: 0.3457 - val_acc: 0.8617
Epoch 45/100
1497/1497 [==============================] - 0s - loss: 0.2816 - acc: 0.8818 - val_loss: 0.3486 - val_acc: 0.8597
Epoch 46/100
1497/1497 [==============================] - 0s - loss: 0.2810 - acc: 0.8784 - val_loss: 0.3463 - val_acc: 0.8557
Epoch 47/100
1497/1497 [==============================] - 0s - loss: 0.2985 - acc: 0.8677 - val_loss: 0.3464 - val_acc: 0.8557
Epoch 48/100
1497/1497 [==============================] - 0s - loss: 0.2824 - acc: 0.8751 - val_loss: 0.3456 - val_acc: 0.85

1497/1497 [==============================] - 0s - loss: 0.1882 - acc: 0.9192 - val_loss: 0.3631 - val_acc: 0.8557
Epoch 70/100
1497/1497 [==============================] - 0s - loss: 0.1774 - acc: 0.9319 - val_loss: 0.3577 - val_acc: 0.8577
Epoch 71/100
1497/1497 [==============================] - 0s - loss: 0.1825 - acc: 0.9185 - val_loss: 0.3552 - val_acc: 0.8517
Epoch 72/100
1497/1497 [==============================] - 0s - loss: 0.1654 - acc: 0.9312 - val_loss: 0.3613 - val_acc: 0.8617
Epoch 73/100
1497/1497 [==============================] - 0s - loss: 0.1749 - acc: 0.9319 - val_loss: 0.3576 - val_acc: 0.8437
Epoch 74/100
1497/1497 [==============================] - 0s - loss: 0.1779 - acc: 0.9272 - val_loss: 0.3625 - val_acc: 0.8577
Epoch 75/100
1497/1497 [==============================] - 0s - loss: 0.1720 - acc: 0.9325 - val_loss: 0.3618 - val_acc: 0.8457
Epoch 76/100
1497/1497 [==============================] - 0s - loss: 0.1711 - acc: 0.9345 - val_loss: 0.3616 - val_acc: 0.84

## Predict Probabilites of each attribute of the given training restaurants

In [17]:
from keras import backend as K

In [22]:
nb_businesses = len(businesses)
nb_attributes = 9

proba_preds = np.empty((nb_businesses, nb_attributes))

In [27]:
for attribute in range(attributes):
    proba_attribute = np.array([]) # initialize attribute probabilities
    for val_fold in range(len(folds)):

        K.clear_session()
        gc.collect()

        # grab the binary model and load the trained weights
        classifier_model = get_binary_vgg16_classifier()
        classifier_model.load_weights(CLASSIFIERS_PATH+'weights/ft_200_attribute_%d_fold_%d.h5' % (attribute, val_fold))

        val_idx = folds[val_fold]
        # grab the businesses_fc1_features as the training set
        X_val = businesses_fc1_features[val_idx]
        
        # predict the probabilty of the attribute for the given classifier 
        predicted_proba = classifier_model.predict_proba(X_val).flatten()
        print(predicted_proba.shape)
        proba_attribute = np.concatenate((proba_attribute, classifier_model.predict_proba(X_val).flatten()))
    
    proba_preds[:, attribute] = (proba_attribute)
np.save(CLASSIFIERS_PATH+'/probas/'+'train_attributes_probas.npy', proba_preds)

 32/499 [>.............................] - ETA: 0s

# Test Set

### Start with the list of business and the FC1 Buisness (see Extract_F1_Features.ipynb)

In [5]:
businesses = np.load(SHARED_PATH+'/test_businesses_shuffled.npy')
businesses_fc1_features = np.load(FEATURES_PATH + 'test_businesses_fc1_blueprint.npy')

# Train classifiers on top of the blueprint bottleneck features per restaurant

In [6]:
from keras import backend as K
from keras import applications
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

Using TensorFlow backend.


### Create a copy of the VGG16 classifier(image net) -> adding dropout and binarizing the output

In [7]:
def get_binary_vgg16_classifier(drop_rate = 0.5):
    
    # Replicate the fc classifier of vgg net
    model = Sequential()
    model.add(Dropout(drop_rate, input_shape=(4096,)))
    model.add(Dense(4096, activation='relu', name='fc2'))
    model.add(Dropout(drop_rate))
    model.add(Dense(1000, activation='relu', name='predictions'))
    
    # Load pre trained weights for image net
    model.load_weights('/home/javier/./.keras/models/vgg16_weights_tf_dim_ordering_tf_kernels.h5', by_name=True)
    
    # Account for the added dropout
    fc2 = model.layers[-3]
    fc2.set_weights([w/drop_rate for w in fc2.get_weights()])
    
    # Remove the predictions layer and add a binary apply/doesen't apply neuron
    model.pop()
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(lr=5e-06), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

## Predict Probabilites of each attribute of the given training restaurants

In [17]:
from keras import backend as K

In [8]:
nb_businesses = len(businesses)
nb_attributes = 9
nb_folds = 4

proba_preds = np.empty((nb_businesses, nb_attributes))

In [11]:
for attribute in range(nb_attributes):
    proba_attribute = np.zeros(nb_businesses) # initialize attribute probabilities
    for val_fold in range(nb_folds):

        K.clear_session()
        gc.collect()

        # grab the binary model and load the trained weights
        classifier_model = get_binary_vgg16_classifier()
        classifier_model.load_weights(CLASSIFIERS_PATH+'weights/ft_200_attribute_%d_fold_%d.h5' % (attribute, val_fold))

        # grab the businesses_fc1_features as the training set
        X_test = businesses_fc1_features
        
        # predict the probabilty of the attribute for the given classifier 
        predicted_proba = classifier_model.predict_proba(X_test).flatten()
        print(predicted_proba.shape)
        proba_attribute += predicted_proba
    
    proba_preds[:, attribute] = (proba_attribute) / 4
np.save(CLASSIFIERS_PATH+'/probas/'+'test_attributes_probas.npy', proba_preds)

10000/10000 [==============================] - 0s     
(10000,)
 9312/10000 [==========================>...] - ETA: 0s(10000,)
